# Figure 1
- schematic
- spectrogram for NSD and SD
- lfp snapshots for sleep classification
- percentage or something for sleep fraction


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from neuropy import plotting
from neuropy.utils import signal_process
from scipy.ndimage import gaussian_filter1d
from scipy import stats
import subjects
import pandas as pd

sd_sessions = subjects.sd.allsess[2:-1]
nsd_sessions =subjects.nsd.ratNday2 + subjects.nsd.ratSday2 + subjects.nsd.ratUday2 
print(sd_sessions)
print(nsd_sessions)

[ProcessData(RatN_Day1_2019-10-09_03-52-32.xml), ProcessData(RatS_Day3SD_2020-11-29_07-53-30.xml), ProcessData(RatR_Day2SD_2021-05-23_09-07-18.xml), ProcessData(RatU_Day4SD_2021-07-29_08-23-06.xml)]
[ProcessData(RatN_Day2_2019-10-11_03-58-54.xml), ProcessData(RatS-Day2NSD-2020-11-27_10-22-29.xml), ProcessData(RatU_Day2NSD_2021-07-24_08-16-38.xml)]


## Calculating spectrogram

In [2]:
spects = []
for sess in [sd_sessions[0],nsd_sessions[0]]:
    signal = sess.eegfile.get_signal([113],t_start=15000,t_stop=17000)
    spects.append(signal_process.SpectrogramBands(signal, window=5, overlap=2))

## Sleep fraction during 5 hours following MAZE

In [3]:
sleep_df = pd.DataFrame()
for grp_ind, sessions in enumerate([sd_sessions, nsd_sessions]):
    for sub, sess in enumerate(sessions):
        maze = sess.paradigm["maze"]
        period = [maze[1], maze[1] + 5 * 3600]
        prop = sess.brainstates.get_proportion_by_label(period[0], period[1])
        if "nan" in prop.keys():
            del prop["nan"]

        if grp_ind == 0:
            grp = "sd"

        if grp_ind == 1:
            grp = "nsd"

        sleep_df = sleep_df.append(
            pd.DataFrame(
                {
                    "prop": list(prop.values()),
                    "state": list(prop.keys()),
                    "grp": grp,
                }
            ),
            ignore_index=True,
        )


## normalized firing rate across session
- normalized firing rate or mean firing rate across population?
- across all states and also in individual states

In [114]:
frate_df = pd.DataFrame()
for neuron_type in ["pyr", "inter"]:
    for sessions, grp in zip([sd_sessions, nsd_sessions], ["sd", "nsd"]):
        for sub, sess in enumerate(sessions):
            pre = sess.paradigm["pre"]
            maze = sess.paradigm["maze"]
            post = sess.paradigm["post"]
            rec_duration = sess.eegfile.duration
            neurons = sess.neurons.get_neuron_type(neuron_type)

            bin_size = 60
            bin_spk_pre = neurons.time_slice(pre[0], maze[1]).get_binned_spiketrains(
                bin_size=bin_size
            )
            bin_spk_post = neurons.time_slice(
                maze[1], maze[1] + 9 * 3600
            ).get_binned_spiketrains(bin_size=bin_size)

            bins = np.arange(-bin_spk_pre.n_bins, bin_spk_post.n_bins)*(60/3600)
            spkcounts = np.hstack((bin_spk_pre.spike_counts, bin_spk_post.spike_counts))
            spkcounts = gaussian_filter1d(spkcounts, sigma=5, axis=-1)
            frate = spkcounts / bin_size
            frate_zscore = stats.zscore(frate, axis=1)
            mean_frate = np.mean(frate_zscore, axis=0)
            # mean_frate = gaussian_filter1d(mean_frate,sigma=2)
            frate_df =frate_df.append(
                pd.DataFrame(
                    {
                        "bins": bins,
                        "frate": mean_frate,
                        "neuron_type": neuron_type,
                        "sub": sub,
                        "grp": grp,
                    }
                ),
                ignore_index=True,
            )


## Plotting

In [129]:
%matplotlib widget
# import matplotlib
# matplotlib.rcParams['text.usetex'] = True
figure = plotting.Fig()
fig,gs = figure.draw(num=1,grid=(4,3),hspace =0.2,wspace=0.35)
fig.suptitle('Figure 1',x=0,ha='left',fontsize=16)

#----- panel a -----------
# ax_exp = plt.subplot(gs[0,:2])
# image = plt.imread('sd_exp_paradigm.png')
# ax_exp.imshow(image,interpolation=None)
# ax_exp.axis('off')
# figure.panel_label(ax_exp,'a')

#------ panel b --------
gs_spect = figure.subplot2grid(gs[1,:],grid=(2,1))

for i, spect in enumerate(spects):
    ax_spect = plt.subplot(gs_spect[i])
    plotting.plot_spectrogram(spect.sxx,spect.time,spect.freq,ax=ax_spect,cmap='plasma',sigma=1,freq_lim=(0,20))
    ax_spect.set_ylabel('Frequency (Hz)')
    ax_theta = ax_spect.twinx()
    ax_theta.plot(spect.time,stats.zscore(spect.theta_delta_ratio),color='w',lw=2)
    ax_theta.set_ylim(top=10)
    ax_theta.set_ylabel(r'($\theta$ \ $\delta$) ratio')
    # plotting.plot_hypnogram(spect.brainstates,ax=ax_spect)

    if i==0:
        figure.panel_label(ax_spect,'B')
        ax_spect.set_xlabel('')
        ax_spect.set_xticklabels([])

#------ panel c --------
ax_sleepfrac = plt.subplot(gs[2,0])
hue = ['active','quiet','rem','nrem']
color = subjects.sleep_colors
sns.barplot(data=sleep_df,x='grp',y='prop',hue='state',ax= ax_sleepfrac,hue_order=hue,palette=subjects.sleep_colors,capsize=0.05)
# ax_sleepfrac.legend([])
ax_sleepfrac.set_ylabel('State proportion')
l = ax_sleepfrac.legend(frameon=False)
for text in l.get_texts():
    text.set_color(subjects.sleep_colors[text.get_text()])

for text in l.legendHandles:
    text.set_visible(False)
figure.panel_label(ax=ax_sleepfrac,label='C')

#------ panel d --------
gs_frate = figure.subplot2grid(gs[2,1:],grid=(1,1),wspace=0.1)
for i,neuron_type in enumerate(['pyr']):
    ax_frate = plt.subplot(gs_frate[i])

    for grp,neuron_color in zip(['sd','nsd'],['#df670c','#633bb5',]):
        df = frate_df[(frate_df['grp']==grp)&(frate_df['neuron_type']==neuron_type)]
        mean_frate = df.groupby('bins').mean()
        x = mean_frate.index
        y = mean_frate.frate.values
        err = df.groupby('bins').sem(ddof=0).frate.values
        ax_frate.fill_between(x,y-err,y+err,color=neuron_color,alpha=0.3,ec=None)
        ax_frate.plot(x,y,neuron_color,lw=1)
        # ax_frate.axvline(0,ls='--')
        # ax_frate.axvline(-60,ls='--',color='r')
        # ax_frate.axvline(300,ls='--',color='k')
        ax_frate.set_xlabel('ZT time (h)')
        ax_frate.set_xlim(left=-4.2)
        # ax_frate.set_ylim([-1.2,1.6])

    ax_frate.axvspan(-4.2,0,0,0.1,color='#6d6d69')
    ax_frate.axvspan(0,9.5,0,0.1,color='#e6e6a2')
    if i==0:
        # ax_frate.axvspan(0,300,0.14,0.2,color='#fa8585')
        ax_frate.set_ylabel('Normalized firing rate')
        # ax_frate.set_xlabel('')
        # ax_frate.set_xticklabels([])
        ax_frate.legend(['SD','NSD'],frameon=False)

        figure.panel_label(ax_frate,'D')

#----- caption --------
ax_caption = plt.subplot(gs[3,:])
ax_caption.text(0,0,'Figure 1: Experimental paradigm, sleep proportion, and firing rate across sessions. \n(A) Experimental paradigm: Following a 3-h baseline recording period (PRE) in their home cage, all rats were transferred to a novel track (MAZE) for task, after which they were returned to their home cage (POST). In POST either animal slept ad libitum (NSD session) or were sleep deprived for 5 h followed by recovery sleep (SD session).\n(B) Example spectrograms of CA1 LFP from SD (top) and NSD (bottom) session. Superimposed white curve represents theta-delta ratio.\n(C) Porportion time spent in each states.\n(D) Mean firing rates across across session pooled across animals. Light/dark cycles are indicated above x-axis.Error bars indicate SEM',wrap=True)
ax_caption.axis('off')

figure.savefig(subjects.fig_folder/'figure1')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=20.0, description='Clim :', max=30.0, min=0.1), IntRangeSlider(value=(…

interactive(children=(FloatSlider(value=20.0, description='Clim :', max=30.0, min=0.1), IntRangeSlider(value=(…